In [1]:
import os, glob, re
from pathlib import Path
from dotenv import load_dotenv
from string import Template
from subprocess import Popen, PIPE

load_dotenv();
assert(os.getenv("PAT"))

In [9]:
def count_active_jobs():
  cmd = "kubectl get jobs"
  stream = os.popen(cmd)
  output = stream.read()
  splits = filter(lambda x : re.search("^j-autoworker-", x), output.split("\n"))
  print(list(splits))
  return len(list(splits))
def get_active_jobs():
  cmd = "kubectl get jobs"
  stream = os.popen(cmd)
  output = stream.read()
  splits = filter(lambda x : re.search("^j-autoworker-", x), output.split("\n"))
  return [q.split()[0] for q in list(splits)]
print(f"Number of active jobs: {count_active_jobs()}")

['j-autoworker-0                             0/1           34s        34s', 'j-autoworker-1                             0/1           33s        33s', 'j-autoworker-2                             0/1           33s        33s', 'j-autoworker-3                             0/1           32s        32s']
Number of active jobs: 0


In [15]:
def create_job(yaml_data):
  p = Popen(["kubectl", "apply", "-f", "-"], shell=True, stdout=PIPE, stderr=PIPE, stdin=PIPE)
  output = p.communicate(input=bytes(yaml_data, encoding="utf-8"))
  output = [line.decode() for line in output if line]
  return output
def kill_job(name):
  p = Popen(["kubectl", "delete", "job", name], shell=True, stdout=PIPE, stderr=PIPE, stdin=PIPE)
  output = p.communicate(input=bytes("", encoding="utf-8"))
  output = [line.decode() for line in output if line]
  return output

In [21]:
configs_path = "smart4.5/configs/clip_2/*"
template_path = "py_template.yml"
pat = os.getenv("PAT")
dry_run = False

for i, path in enumerate(glob.glob(configs_path)):
  path = Path(path)
  path_small = Path(*path.parts[1:])
  if dry_run:
    print("dry_run", path)
  else:
    with open(template_path, "r") as f:
      contents = f.read()
      template = Template(contents)
      template = template.safe_substitute(pat=pat, config=path_small.as_posix(), i=i)
      output = create_job(template)
      print(f"==={template_path}===")
      # print(template)
      print("\t", output)
      

===py_template.yml===
	 ['job.batch/j-autoworker-0 created\n']
===py_template.yml===
	 ['job.batch/j-autoworker-1 created\n']
===py_template.yml===
	 ['job.batch/j-autoworker-2 created\n']
===py_template.yml===
	 ['job.batch/j-autoworker-3 created\n']


In [22]:
jobs = get_active_jobs()
print(jobs)
dry_run = False
for job in jobs:
  print(f"Job: {job}")
  if not dry_run:
    kill_job(job)